In [1]:
import sys
project_root_path = '../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_stats

In [2]:
import json
in_path = project_root_path + '/python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

cfg = config_map['99_problems']

In [3]:

path = project_root_path + '/' + cfg['stats_dir']
all_commit_stats = commit_stats.load_commit_stats(cfg['stat_file_name'], path)
all_commit_stats[slice(0,2)]

[{'commit': '44ed13963ea38e665d6bd93779e76298334375d5',
  'Author': 'mayeut <mayeut@users.noreply.github.com>',
  'Date': 'Date:   Mon May 16 11:02:03 2022 +0200',
  'fileTypes': {'csv': {'inserts': 1380, 'deletes': 0, 'occurrences': 1}}},
 {'commit': '401ff2d998460abd1554c3cd475ebc9192a271d5',
  'Author': 'mayeut <mayeut@users.noreply.github.com>',
  'Date': 'Date:   Sun May 15 22:18:57 2022 +0000',
  'fileTypes': {'json': {'inserts': 2, 'deletes': 0, 'occurrences': 1}}}]

In [4]:
ft_dicts = []
ft_dict_dict = {}
for stats in all_commit_stats:
    for file_type in stats['fileTypes']:
        commit_ft_dict = stats['fileTypes'][file_type]
        if file_type not in ft_dict_dict:
            ft_dict = {}
            ft_dicts.append(ft_dict)
            ft_dict_dict[file_type] = ft_dict
            ft_dict['type'] = file_type
            ft_dict['lines'] = 0
            ft_dict['files'] = 0
            ft_dict['commits'] = 0
        ft_dict = ft_dict_dict[file_type]
        ft_dict['commits'] += 1
        ft_dict['lines'] += commit_ft_dict['inserts']
        ft_dict['files'] += commit_ft_dict['occurrences']


In [5]:
by_commit_count = sorted(ft_dicts, key=lambda d: d['commits'])
by_commit_count.reverse()
# if your odds of being in a commit are < 1/1000, you don't matter
main_type_stats = [stats for stats in by_commit_count if stats['commits'] > (len(all_commit_stats) / 1000)]
print(len(main_type_stats))
main_type_stats

70


[{'type': 'py', 'lines': 13394375, 'files': 553574, 'commits': 206185},
 {'type': 'rst', 'lines': 929207, 'files': 63762, 'commits': 38783},
 {'type': 'js', 'lines': 16373937, 'files': 208292, 'commits': 38307},
 {'type': 'md', 'lines': 4194152, 'files': 197874, 'commits': 37792},
 {'type': 'json', 'lines': 52971503, 'files': 149520, 'commits': 36875},
 {'type': 'noexttext', 'lines': 1574100, 'files': 77394, 'commits': 30601},
 {'type': 'ts', 'lines': 2300455, 'files': 126613, 'commits': 25149},
 {'type': 'txt', 'lines': 1012771, 'files': 40266, 'commits': 24610},
 {'type': 'cpp', 'lines': 1141422, 'files': 46877, 'commits': 21658},
 {'type': 'tsx', 'lines': 823165, 'files': 58325, 'commits': 17008},
 {'type': 'html', 'lines': 3487556, 'files': 76212, 'commits': 15978},
 {'type': 'yml', 'lines': 244643, 'files': 20013, 'commits': 14938},
 {'type': 'h', 'lines': 1013885, 'files': 30206, 'commits': 11269},
 {'type': 'go', 'lines': 8284968, 'files': 77711, 'commits': 11077},
 {'type': 'lo

In [6]:
import pandas as pd

main_types = [stat['type'] for stat in main_type_stats]
main_types.index('json')
rows = []
for commit_stats in all_commit_stats:
    row = [0]*len(main_types)
    rows.append(row)
    for file_type in commit_stats['fileTypes']:
        commit_ft_dict = commit_stats['fileTypes'][file_type]
        if file_type in main_types:
            position = main_types.index(file_type)
            row[position] = commit_ft_dict['inserts']
type_df = pd.DataFrame(rows, columns=main_types) # index=commit_ids,
type_df

,py,rst,js,md,json,noexttext,ts,txt,cpp,tsx,...,doc,snap,jpg,tex,template,php,svelte,removal,ipynb,list
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476275,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476276,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476277,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
type_corr = type_df.corr()
print(str(type_corr))

               py       rst        js        md      json  noexttext  \
py       1.000000  0.024582  0.004519  0.000955 -0.000281   0.002367   
rst      0.024582  1.000000  0.001717 -0.000364 -0.000702   0.002408   
js       0.004519  0.001717  1.000000  0.820101  0.152648   0.163104   
md       0.000955 -0.000364  0.820101  1.000000  0.132963   0.143746   
json    -0.000281 -0.000702  0.152648  0.132963  1.000000   0.026346   
...           ...       ...       ...       ...       ...        ...   
php      0.000565 -0.000554  0.027707  0.000658  0.000389   0.002268   
svelte  -0.000521 -0.000495 -0.000002 -0.000109  0.001191  -0.000034   
removal  0.000233  0.007408 -0.000147 -0.000309 -0.000328  -0.000085   
ipynb    0.003759  0.005264 -0.000034 -0.000066 -0.000068  -0.000013   
list    -0.000300 -0.000273 -0.000034 -0.000022 -0.000076  -0.000016   

               ts       txt       cpp       tsx  ...       doc      snap  jpg  \
py      -0.000456  0.010008  0.004661 -0.002190  ... -

In [12]:
for i in range(len(main_types)):
    for j in range(len(main_types)):
        if i < j:
            val = type_corr.iat[i, j]
            if val > 0.2:
                print(main_types[i] + ", " + main_types[j] + ": " + str(val))

js, md: 0.8201010709097677
js, ts: 0.48542285785509237
js, txt: 0.5758181423810181
js, html: 0.28273231466687576
js, h: 0.34264959582191784
js, lock: 0.23618049995157184
js, sol: 0.4727662446804384
js, in: 0.4788243480174249
js, svg: 0.507499917963075
js, patch: 0.2682122739521914
js, snap: 0.298392890898753
md, ts: 0.4173427416581278
md, txt: 0.5086853906097407
md, html: 0.24567794725568712
md, h: 0.3024628152002233
md, sol: 0.4095310246822728
md, in: 0.41195849025514275
md, svg: 0.4362001420214794
md, patch: 0.23057283515412338
md, snap: 0.2562638880740277
ts, txt: 0.2933761332328184
ts, sol: 0.2408013335087905
ts, in: 0.2445244695923032
ts, svg: 0.2583505602657806
txt, h: 0.2226551937349764
txt, go: 0.24089759670891986
txt, sol: 0.2837374706937876
txt, in: 0.2925657265742262
txt, svg: 0.30967070025090443
txt, gz: 0.24204869852774252
txt, tex: 0.3872012411854471
cpp, hpp: 0.4868760937120625
cpp, xml: 0.37954617423438997
h, sh: 0.21902102981409935
h, ini: 0.3206697285734677
h, swift: 